## Includes while package stuff is sorted out

In [1]:
using JuMP
using MathProgBase.SolverInterface
using GLPKMathProgInterface
using LazySets

In [2]:
include("../src/utils/activation.jl")
include("../src/utils/network.jl")
include("../src/utils/problem.jl")
include("../src/utils/util.jl")

include("../src/reachability/maxSens.jl")

partition (generic function with 2 methods)

# Set-up vars

In [31]:
small_nnet = read_nnet("../examples/networks/small_nnet.txt")

A = Array{Float64, 2}(2,1)
A[1,1] = 1
A[2,1] = -1
#inputSet = HPolytope(A, [1.0,3.0])
#outputSet = HPolytope(A,[0.0,1000.0])

inputSet = Hyperrectangle([1.0, 1.0], [.2, .2])
inputSet = Hyperrectangle([2.0,], [100.0])

problem = Problem(small_nnet, inputSet, outputSet)
solver = GLPKSolverLP(method=:Exact)
m = Model(solver=solver)

Feasibility problem with:
 * 0 linear constraints
 * 0 variables
Solver is GLPKInterfaceLP

### Test variables

In [32]:
function init_nnet_vars(model::Model, network::Network)
    layers = network.layers
    #input layer and last layer have b_vars because they are unbounded
    b_vars = Vector{Vector{Variable}}(length(layers) + 1) # +1 for input layer
    #f_vars are always positive and used as front for ReLUs
    f_vars = Vector{Vector{Variable}}(length(layers) -1)
    
    input_layer_n = size(first(layers).weights, 2)
    all_layers_n  = [length(l.bias) for l in layers]
    insert!(all_layers_n, 1, input_layer_n)

    for (i, n) in enumerate(all_layers_n)
        b_vars[i] = @variable(model, [1:n]) # To do: name the variables
        if 1 < i < length(layers) + 1
            f_vars[i-1] = @variable(model, [1:n])
        end
    end
    return b_vars, f_vars
end

init_nnet_vars (generic function with 1 method)

## Functions that were not included 

In [33]:
function add_input_constraint(model::Model, input::HPolytope, neuron_vars::Vector{Variable})
    in_A,  in_b  = tosimplehrep(input)
    @constraint(model,  in_A * neuron_vars .<= in_b)
    return nothing
end

function add_input_constraint(model::Model, input::Hyperrectangle, neuron_vars::Vector{Variable})
    @constraint(model,  neuron_vars .<= high(input))
    @constraint(model,  neuron_vars .>= low(input))
    return nothing
end

function add_output_constraint(model::Model, output::AbstractPolytope, neuron_vars::Vector{Variable})
    out_A, out_b = tosimplehrep(output)
    @constraint(model, out_A * neuron_vars .<= out_b)
    return nothing
end

add_output_constraint (generic function with 1 method)

In [58]:
function encode(model::Model, problem::Problem, relustatus::Array{Array{Int64,1},1})
    #bs, fs = init_nnet_vars(model, problem.network)
    
    #TEST
    bs = [[@variable(m, b11)],
         [@variable(m, b21),@variable(m, b22)],
         [@variable(m, b31),@variable(m, b32)],  
         [@variable(m, b41)]]

    fs = [[@variable(m, f21),@variable(m, f22)],
          [@variable(m, f31),@variable(m, f32)]]
    
    #add_input_constraint(model, inputSet, bs[1])
    #add_output_constraint(model, problem.output, last(bs))

    for (i, layer) in enumerate(problem.network.layers)
        #print(string("i: ",i))
        (W, b, act) = (layer.weights, layer.bias, layer.activation)
        #before_act = W * neurons[i] + b
        
        #first layer is different
        if i == 1
            for j in 1:length(layer.bias)
               @constraint(model, -bs[2][j] + bs[1][1]*W[j] == -b[j]) 
            end
        elseif 1<i
            for j in 1:length(layer.bias) # For evey node
                @constraint(model, -bs[i+1][j] + dot(fs[i-1],W[j,:]) == -b[j])
            end
        end
    end

    # Adding linear constraints
    
    #first layer
    bounds = get_bounds(problem)
    
    
    #@constraint(model, bs[1] .<= bounds[1].center + bounds[1].radius)
    #@constraint(model, bs[1] .>= bounds[1].center - bounds[1].radius)
    
    for i in 1:length(bs)
        @constraint(model, bs[i] .<= bounds[i].center + bounds[i].radius)
        @constraint(model, bs[i] .>= bounds[i].center - bounds[i].radius)
    end
    
    # positivity contraint for f variables
    for i in 1:length(fs)
        @constraint(model, fs[i] .>= zeros(length(fs[i])))
    end
    # Objective: L∞ norm of the disturbance\
    @objective(m, Max, 0)
    return (bs, fs)
end

encode (generic function with 1 method)

In [42]:
get_bounds(problem)[4]

LazySets.Hyperrectangle{Float64}([1260.5], [1242.0])

In [43]:
get_bounds(problem)[4].center + get_bounds(problem)[4].radius

1-element Array{Float64,1}:
 2502.5

In [44]:
get_bounds(problem)[4].center -  get_bounds(problem)[4].radius

1-element Array{Float64,1}:
 18.5

In [59]:
m = Model(solver = GLPKSolverLP(method=:Exact))
#bs, fs = init_nnet_vars(m, problem.network)
bs, fs = encode(m, problem, [[1,1], [1,1]])

(Array{JuMP.Variable,1}[JuMP.Variable[b11], JuMP.Variable[b21, b22], JuMP.Variable[b31, b32], JuMP.Variable[b41]], Array{JuMP.Variable,1}[JuMP.Variable[f21, f22], JuMP.Variable[f31, f32]])

In [60]:
m

Maximization problem with:
 * 21 linear constraints
 * 10 variables
Solver is GLPKInterfaceLP

In [61]:
JuMP.solve(m)

:Optimal

In [62]:
bs[1]

1-element Array{JuMP.Variable,1}:
 b11

In [80]:
b_test[2:length(b_test)-1]

2-element Array{Array{Float64,1},1}:
 [0.0, 0.0]
 [2.5, 2.5]

In [146]:
function check_relu_status(bs::Array{Array{JuMP.Variable,1},1}, 
        fs::Array{Array{JuMP.Variable,1},1})
    b_values = [getvalue(b) for b in bs[2:length(bs)-1]]
    f_values = [getvalue(f) for f in fs]
    return [x[1] .== x[2] for x in zip(b_values, f_values)]
end

check_relu_status (generic function with 1 method)

In [ ]:
function fix_broken_relu

In [ ]:
function reluplexrun